In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
#Sample rentals.ca website
rentals = "https://rentals.ca/toronto?p=2"
response = get(rentals, headers=headers)
#200 is a good response
print(response)

<Response [200]>


In [3]:
def scrapping_rentals_ca(pagenum = 50):
    #Initialize the final dict for return
    dic = {'name':[], 'price': [], 'latitude': [], 'longitude': [], 'url': [], 'location': [], 'room_type': []}
    k=1
    new_url=""
    #Initialize the link
    initial_link = 'https://rentals.ca/toronto?p='
    #Go to the specificied link page and get the housing info
    for idx in range(pagenum):
        link = initial_link + str(idx+1)
        response = get(link, headers=headers)
        html_soup = BeautifulSoup(response.text, 'html.parser')

        all_script = html_soup.find_all('script')

        json_list = []
        for item in all_script:
            if str(item).startswith('''<script type="applic'''):
                json_list.append(str(item))
        
        #For each webpage, save its info in a temp list and then add to dictionary
        for item in json_list:
            i = 1
            j = 1
            temp_room_type = []
            temp_price = []
            temp_name = []
            for words in item.split('\n'):
                if words.strip()[:6] == '''"name"''':
                    if i == 1:
                        temp_name.append(words.strip()[7:-1].strip().strip('\"'))
                        i += 1
                    elif i == 2:
                        dic['location'].append(''.join(e for e in words.strip()[7:-1] if e.isalnum()))
                        i += 1
                    else:
                        temp_room_type.append(words.strip()[7:-1].strip().strip('\"'))
                        
                if words.strip()[:7] == '''"price"''':
                    temp_price.append(float(words.strip()[8:].strip()[:-1]))
                if words.strip()[:10] == '''"latitude"''':
                    dic['latitude'].append(float(words.strip()[11:].strip()[:-1]))
                if words.strip()[:11] == '''"longitude"''':
                    dic['longitude'].append(float(words.strip()[12:].strip()[:-1]))
                if words.strip()[:5] == '''"url"''' and j == 1:
                    dic['url'].append(words.strip()[6:-1].strip().strip('\"'))
                    j += 1
            if len(temp_room_type) == 0 and len(temp_price) == 0:
                if len(temp_name) != 0:
                    temp_name.pop()
            else:
                dic['name'].append(temp_name)
                dic['room_type'].append(temp_room_type)
                dic['price'].append(temp_price)
                #print("Iteration {}, dictionary length now is {}".format(idx, [len(x) for x in dic.values()]))
                
            if len(dic['url'])!=0 and dic['url'][-1]!=new_url:
                sub_name=dic['name'][-1]
                print(sub_name)
                new_url=dic['url'][-1]
                print(new_url)

                #Extract from new_url
                sub_response = get(new_url, headers=headers)

                sub_html_soup = BeautifulSoup(sub_response.text, 'html.parser')

                sub_all_script = sub_html_soup.find_all('script')

                sub_json_list = []
                for sub_item in sub_all_script:
                    if str(sub_item).startswith('''<script type="text/javascript"'''):
                        sub_json_list.append(str(sub_item))

                start=sub_json_list[1].find("raw_amenities")
                end=sub_json_list[1].find("categories")

                sub_feature=sub_json_list[1][start:end]

                first_split=sub_feature.split('"name": ')
                second_split=[x.split(', "slug"')[0] for x in first_split]
                sub_feature_info=[x for x in second_split if x not in ['"Building Features"','"Unit Features"','"Utilities"']][1:]

                feature_df=pd.DataFrame(sub_feature_info,columns = ['Features'] )
                sub_feature_info=pd.get_dummies(feature_df['Features']).max(axis=0).to_frame().T
                sub_feature_info['name'] = sub_name 
                if k == 1:
                    final_sub_feature=sub_feature_info
                    k += 1
                else:
                    frames = [final_sub_feature, sub_feature_info]
                    final_sub_feature=pd.concat(frames)
    
    return dic,final_sub_feature


In [4]:
scrapped,final_sub_feature = scrapping_rentals_ca(pagenum=3)

['120 Raglan Avenue']
https://rentals.ca/york/120-raglan-avenue
['310-312 Lonsdale Road']
https://rentals.ca/toronto/310-312-lonsdale-road
['35 Walmer Road']
https://rentals.ca/toronto/35-walmer-road
['50 Spadina Road']
https://rentals.ca/toronto/50-spadina-road
['45 Carlton Street']
https://rentals.ca/toronto/45-carlton-street-2
['191 Sherbourne Street']
https://rentals.ca/toronto/sherbourne-estates-1
['201 Sherbourne Street']
https://rentals.ca/toronto/sherbourne-estates-2
['Goldengate Apartments']
https://rentals.ca/north-york/1790-eglinton-avenue-east
['730 Dovercourt Road - Doversquare Apartments']
https://rentals.ca/toronto/730-dovercourt-road-doversquare-apartments
['2900 Bathurst Street - Emerald Towers']
https://rentals.ca/north-york/2900-bathurst-street-emerald-towers
['4001 Bayview Avenue - The Gates of Bayview']
https://rentals.ca/north-york/4001-bayview-avenue-the-gates-of-bayview
['18 BROWNLOW']
https://rentals.ca/toronto/18-brownlow-ave-1
['650 Parliament Street']
https:

In [5]:
scrapped_df= pd.DataFrame.from_dict(scrapped)

In [6]:
scrapped_df.name = scrapped_df.name.apply(lambda x: x[0])

In [7]:
final_table=pd.merge(scrapped_df,final_sub_feature,on='name')

In [8]:
final_table

,name,price,latitude,longitude,url,location,room_type,"""Balcony""","""Cable Ready""","""Convenience Store""",...,"""Guest Suite""","""Rooftop Garden""","""Rooftop Lounge""","""Theatre Room""","""Furnished""","""Newly Renovated""","""freezer included""","""fridge included""","""oven included""","""smoker friendly"""
0,120 Raglan Avenue,"[1864.0, 2003.0, 2347.0]",43.686516,-79.421005,https://rentals.ca/york/120-raglan-avenue,York,"[Jr. 1 Bedroom, 1 Bedroom, 2 Bedroom]",1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,310-312 Lonsdale Road,"[2134.0, 2475.0]",43.689224,-79.411199,https://rentals.ca/toronto/310-312-lonsdale-road,Toronto,"[1 Bedroom, 2 Bedroom]",NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,35 Walmer Road,"[2183.0, 2353.0, 2888.0]",43.668877,-79.405555,https://rentals.ca/toronto/35-walmer-road,Toronto,"[Jr. 1 Bedroom, 1 Bedroom, 2 Bedroom]",1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50 Spadina Road,"[2549.0, 2732.0, 3506.0]",43.669070,-79.404672,https://rentals.ca/toronto/50-spadina-road,Toronto,"[1 Bedroom, Large 1-Bedroom, 2 Bedroom]",1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45 Carlton Street,[3700.0],43.661415,-79.380663,https://rentals.ca/toronto/45-carlton-street-2,Toronto,[2.5 Bedroom],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,191 Sherbourne Street,"[1749.0, 1879.0, 2329.0]",43.656741,-79.369994,https://rentals.ca/toronto/sherbourne-estates-1,Toronto,"[1 Bedroom Med Renovated, 1 Bedroom Large Reno...",1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201 Sherbourne Street,"[1749.0, 1879.0, 2249.0]",43.657668,-79.370436,https://rentals.ca/toronto/sherbourne-estates-2,Toronto,"[1 Bedroom Med Renovated, 1 Bedroom Large Reno...",1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Goldengate Apartments,"[1700.0, 1750.0, 2100.0]",43.724783,-79.302956,https://rentals.ca/north-york/1790-eglinton-av...,NorthYork,"[One Bedroom, One Bedroom (Large), Two Bedroom]",1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,730 Dovercourt Road - Doversquare Apartments,"[1707.0, 1887.0, 1907.0]",43.660713,-79.429237,https://rentals.ca/toronto/730-dovercourt-road...,Toronto,"[Bachelor, Jr. 1 Bedroom, 1 Bedroom]",1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2900 Bathurst Street - Emerald Towers,"[2347.0, 2747.0]",43.714525,-79.428731,https://rentals.ca/north-york/2900-bathurst-st...,NorthYork,"[2 Bedroom, 3 Bedroom]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Columns: 122 entries, name to "smoker friendly"
dtypes: float64(117), object(5)
memory usage: 28.8+ KB


In [10]:
csv_columns = list(final_table.columns.values)
csv_columns

['name',
 'price',
 'latitude',
 'longitude',
 'url',
 'location',
 'room_type',
 '"Balcony"',
 '"Cable Ready"',
 '"Convenience Store"',
 '"Dishwasher Available"',
 '"Flooring - Ceramic"',
 '"Flooring - Hardwood"',
 '"Fridge"',
 '"Heating"',
 '"Laundry Facilities"',
 '"Microwave"',
 '"Nearby"',
 '"On-site Management"',
 '"On-site Staff"',
 '"Parking - Underground"',
 '"Parking - Visitor"',
 '"Parks"',
 '"Public Transit"',
 '"Schools"',
 '"Shopping"',
 '"Storage Lockers"',
 '"Stove"',
 '"Walk-in Closet"',
 '"Water"',
 '"Other"',
 '"Outdoor Parking"',
 '"Air Conditioning"',
 '"Elevators"',
 '"Fitness Room"',
 '"Keyless Entry"',
 '"Recreation Room"',
 '"Social Room"',
 '"Blinds"',
 '"Dryer"',
 '"Internet Ready"',
 '"Sauna"',
 '"Washer"',
 '"24h Security"',
 '"Bike Racks"',
 '"Central Air Conditioning"',
 '"Elevator"',
 '"Exercise Room"',
 '"Fitness Area"',
 '"Hydro / Electricity"',
 '"Jacuzzi"',
 '"Party Room"',
 '"Pool - Heated"',
 '"Rooftop Deck"',
 '"Secured Access"',
 '"Security On-si

In [11]:
final_table.to_csv('rental.csv', index=False)